In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import os

#open csv
filepath = os.path.join('..', 'Database', 'TBIMSPublic.2024-11-01', 'Data', 'Form2.csv')

df = pd.read_csv(filepath)

/var/folders/_6/mkj9qwf17jbfrz94kdykryqh0000gn/T/ipykernel_48516/2752882984.py:10: DtypeWarning: Columns (5,6,49,52,53) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(filepath)


In [ ]:
#inspect
df.info()
print(df.notnull().sum())
df.describe(include="all")

In [13]:
#inspect mixed columns index
mixed_type_col = [15,6,49,52,53]

#find column names
mixed_type_col_names =[df.columns[column] for column in mixed_type_col]

print(mixed_type_col_names)
print(df['DeathCause2F'].value_counts(dropna=False))


['PRTSchoolF', 'DeathCause2F', 'Rehosp1lv2F', 'Rehosp4lv2F', 'Rehosp5lv2F']
DeathCause2F
88888.0    58305
99999.0     9120
88888       4237
99999       1228
44444.0      513
           ...  
402.91         1
701.0          1
038.9          1
429.2          1
C34.90         1
Name: count, Length: 411, dtype: int64


In [23]:
form_vars_filepath = os.path.join('..', 'Database', 'TBIMSPublic.2024-11-01', 'Code', 'Form2_Variables.csv')
form_vars_df = pd.read_csv(form_vars_filepath)
form_vars_df['Name']

0                      Mod2Id
1                      Mod1Id
2              FollowUpPeriod
3                   IntStatus
4                 LostReasonF
                ...          
307                   SRScalF
308                 SWLSTOT4F
309                  SWLSTOTF
310    WordRecallCorrectF_i_n
311       LastValidFollowupYN
Name: Name, Length: 312, dtype: object

In [ ]:
var_codes_filepath = os.path.join('..', 'Database', 'TBIMSPublic.2024-11-01', 'Code', 'Form2_Codes.csv')


df_var_codes = pd.read_csv(var_codes_filepath)

var_labels_dict ={}
for var in df_var_codes['VariableName'].unique():
    sub_df = df_var_codes[df_var_codes["VariableName"] == var]
    value_map = dict(zip(sub_df['Code'], sub_df['CodeDescription']))
    var_labels_dict[var] = value_map

In [18]:
df.dtypes.value_counts()

float64    283
int64       21
object       8
Name: count, dtype: int64

In [28]:
def normalize_code(code):
    """ will convert to format 'string.0' """
    #try to convert to a float directly
    try:
        return str(float(code))
    #if it can't be converted to a float, save it as a string
    except ValueError:
        return str(code)

def process_mixed_variables(df, value_labels_dict, replace_col=False, code_col=False, code_label_col=False,errors="coerece"):
    """
    Cleans coded categorical/numeric variables using a value label dict.
    Returns a cleaned dataframe and a summary.
    """
    df = df.copy()
    original_cols = df.columns
    col_map = {col.lower(): col for col in original_cols}

    new_columns = {}
    summary_list = []


    for var_raw, codes_dict in value_labels_dict.items():
        var_lower = var_raw.lower()

        if var_lower not in col_map:
            continue

        original_var = col_map[var_lower]

        # Work on a lowercase temporary series for logic
        temp_series = df[original_var].copy()

        # Normalize for mapping
        code_keys = set(normalize_code(k) for k in codes_dict.keys())

        # Convert entries for numeric masking
        temp_series = temp_series.apply(lambda x: str(float(x)) if pd.notna(x) and str(x).replace('.', '', 1).isdigit() else x)
        clean_series = pd.to_numeric(temp_series.mask(temp_series.isin(code_keys)), errors=errors)

        mapped_dict = {str(float(k)) if k.isdigit() else k: v for k, v in codes_dict.items()}
        label_series = temp_series.map(mapped_dict)

        # Output columns using original name
        if replace_col:
            new_columns[original_var] = clean_series
        else:
            new_columns[f'{original_var}_clean'] = clean_series

        if code_col:
            new_columns[f'{original_var}_code'] = temp_series.where(temp_series.isin(code_keys))
        
        if code_label_col:
            new_columns[f'{original_var}_code_label'] = label_series

        summary = {
            "Variable": original_var,
            "Valid Count": clean_series.count(),
            "Mean": clean_series.mean(),
            "Std Dev": clean_series.std(),
            "all_categorical": clean_series.isna().all()
        }

        summary.update(label_series.value_counts().to_dict())
        summary_list.append(summary)
    

    # Drop old versions of columns being replaced
    df = df.drop(columns=new_columns.keys(), errors='ignore')
    df = pd.concat([df, pd.DataFrame(new_columns)], axis=1)
    summary_df = pd.DataFrame(summary_list).infer_objects(copy=False)

    return df, summary_df

In [ ]:
df_decoded, summary = process_mixed_variables(df, var_labels_dict, replace_col=True,code_col=True,code_label_col=True,errors="coerce")

col_map = {col.lower(): col for col in df.columns} 

categorical_cols = summary[summary['all_categorical'] == True]['Variable'].str.lower().str.strip().tolist()

continuous_cols = [
    i.lower().strip()
    for i in form_vars_df['Name'].unique()
    if i.lower().strip() not in categorical_cols
]

continuous_cols = list(map(col_map.get, continuous_cols))



['Mod2Id', 'Mod1Id', 'FollowUpPeriod', 'DeathCause1F', 'DeathCause2F', 'DeathECodeF', 'Rehosp1lv2F', 'Rehosp2lv2F', 'Rehosp3lv2F', 'Rehosp4lv2F', 'Rehosp5lv2F', 'HeightF', 'WeightF', 'ALCWeekF', 'ALCDrinksF', 'ALC5DrinksF', 'ALC4DrinksF', 'drs3_1F', 'WordRecallPrimacyF', 'WordRecallMiddleF', 'WordRecallRecencyF', 'WordRecallCorrectF', 'WordRecallRepF', 'WordRecallIntF', 'BackDigitCorrectF', 'FluencyCorrect1_15F', 'FluencyCorrect15_30F', 'FluencyCorrect30_45F', 'FluencyCorrect45_60F', 'FluencyCorrectF', 'FluencyRepF', 'FluencyIntF', 'ReasonCorrectF', 'BackCountLastNumF', 'BackCountErrorsF', 'BackCountDigitsF', 'BackCountTimeF', 'DelayWordRecallPrimacyF', 'DelayWordRecallMiddleF', 'DelayWordRecallRecencyF', 'DelayWordRecallCorrectF', 'DelayWordRecallRepF', 'DelayWordRecallIntF', 'LengthInterviewF', 'DRSEmpF', 'DRSFeedF', 'DRSToiletF', 'DRSGroomF', 'DRSFuncF', 'SchoolF', 'ALCMonthF', 'AGENoPHIF', 'B3TCOMPF', 'B3TEFF', 'B3TEMF', 'BackCountDigitsF_i_n', 'BackDigitCorrectF_i_n', 'BMIF', 'Com

In [32]:
df_decoded[continuous_cols].astype(float)

ValueError: could not convert string to float: 'PI_Original'

In [31]:
#Compute IQR

data = df_decoded[continuous_cols]


Q1 = data.quantile(0.25)
Q3 = data.quantile(0.75)
IQR = Q3 - Q1

# Define bounds (you can adjust the multiplier)
lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

# Filter out outliers
filtered_data = data[(data >= lower_bound) & (data <= upper_bound)]



# Plot histogram

ax = filtered_data.hist(figsize=(20, 17))

# Adjust spacing between subplots
plt.subplots_adjust(hspace=0.6, wspace=0.7)
plt.title("Histogram (Outliers Removed)")
plt.show()

TypeError: unsupported operand type(s) for -: 'str' and 'str'